# Shark League Score
## Import Python libraries

In [30]:
import importlib
import sys

# Remove cached modules to force reload
if 'swc_simple_client' in sys.modules:
    del sys.modules['swc_simple_client']

import pandas as pd
import numpy as np
import logging
import swc_simple_client as swc


## Configure logging

In [31]:
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(filename='shark_notebook.log', level=logging.INFO,)

## Setup notebook variables

In [32]:
base_url = "https://super-duper-palm-tree-g4v9vrvxx9vgc9jqv-8000.app.github.dev"

## Get Max Scores
### USe endpoint: LIST_WEEKS_ENDPOINT

In [34]:
week_api_response = swc.call_api_endpoint(base_url, swc.LIST_WEEKS_ENDPOINT)
print(f"Response status code: {week_api_response.status_code}")
print(f"Response content-type: {week_api_response.headers.get('content-type', 'N/A')}")

if week_api_response.status_code == 200:
    # Check if response is actually JSON
    content_type = week_api_response.headers.get('content-type', '')
    if 'application/json' in content_type:
        weeks_df = pd.DataFrame(week_api_response.json())
        weeks_df['year'] = weeks_df['week_number'].str.slice(0,4).astype(int)

        max_totals_grouped_df = weeks_df.groupby('year').agg(
            ppr_12_max_points=('ppr_12_total_points', 'sum'),
            half_ppr_8_max_points=('half_ppr_8_total_points', 'sum'))

        display(max_totals_grouped_df)
        logging.info("Max total points per scoring format per year calculated.")
    else:
        print(f"ERROR: Response is not JSON. Content-Type: {content_type}")
        print(f"Response text (first 500 chars): {week_api_response.text[:500]}")
else:
    print(f"API Error: {week_api_response.status_code}")
    print(f"Response text (first 500 chars): {week_api_response.text[:500]}")


Response status code: 200
Response content-type: text/html; charset=utf-8
ERROR: Response is not JSON. Content-Type: text/html; charset=utf-8
Response text (first 500 chars): <!doctype html><html lang="en"><head><meta charset="UTF-8"/><meta name="authUrl" content="https://github.com/codespaces/auth/super-duper-palm-tree-g4v9vrvxx9vgc9jqv"/><meta name="gitHubApiUrl" content="https://api.github.com"/><meta name="correlationId" content="b30240cc-45f7-450b-a20e-6e746ffcfa1e"/><meta name="iKey" content="f772ffaa012e4fc6bb0a245dd176fc6c-ca6358be-0b85-4e74-ade1-c7857dd7d8c9-7394"/><link id="js-favicon" rel="shortcut icon" href="https://github.githubassets.com/favicon.ico"/>
